<a href="https://colab.research.google.com/github/R-Saad007/Streamlit-Mistral-7B-GPT/blob/main/BrightSitesGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.457s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [2]:
%%writefile requirements.txt
langchain
torch
accelerate
sentence_transformers
streamlit_chat
streamlit
faiss-cpu
tiktoken
huggingface-hub
pypdf
llama-cpp-python

Overwriting requirements.txt


In [3]:
!pip install -r requirements.txt
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 55.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5

In [4]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
%%writefile BrightSitesGPT.py

import streamlit as st
st.set_page_config(layout="wide")
from streamlit_chat import message
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
import os
import tempfile
from PIL import Image

def initialize_session_state():
  if 'l' not in st.session_state:
      st.session_state['history'] = []

  if 'generated' not in st.session_state:
      st.session_state['generated'] = ["Hey! How may I help you today?"]

  if 'past' not in st.session_state:
      st.session_state['past'] = ['Hi!']

def conversation_chat(query, chain, history):
  result = chain({"question": query, "chat_history": history})
  history.append((query, result["answer"]))
  return result["answer"]

def display_chat_history(chain):
  reply_container = st.container()
  container = st.container()

  with container:
      with st.form(key='my_form', clear_on_submit=True):
          user_input = st.text_input("Question:", placeholder="What do you want to learn about BrightSites?", key='input')
          submit_button = st.form_submit_button(label='Submit')

      if submit_button and user_input:
          with st.spinner('Generating response...'):
              output = conversation_chat(user_input, chain, st.session_state['history'])

          st.session_state['past'].append(user_input)
          st.session_state['generated'].append(output)

  if st.session_state['generated']:
      with reply_container:
          for i in range(len(st.session_state['generated'])):
                  message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="adventurer")
                  message(st.session_state["generated"][i], key=str(i), avatar_style="bottts")

@st.cache_resource(show_spinner=False)
def load_embeddings():
  return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                    model_kwargs={'device': 'cpu'})

@st.cache_resource(show_spinner=False)
def load_vectorstore(_text_chunks, _embeddings):
  return FAISS.from_documents(_text_chunks, embedding=_embeddings)

@st.cache_resource(show_spinner=False)
def load_model(_vector_store):
  #Import Model
  llm = LlamaCpp(
  streaming = True,
  model_path="/content/drive/MyDrive/Model/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
  temperature=0.75,
  top_p=0.5,
  top_k=0.5,
  verbose=True,
  n_gpu_layers=60,
  n_ctx=4096
)
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
  chain = ConversationalRetrievalChain.from_llm(llm=llm, chain_type='stuff',
                                                retriever=_vector_store.as_retriever(search_kwargs={"k": 2}),
                                                memory=memory)
  return chain


def processing(uploaded_files):
  text = []
  for file in uploaded_files:
      file_extension = os.path.splitext(file.name)[1]
      with tempfile.NamedTemporaryFile(delete=False) as temp_file:
          temp_file.write(file.read())
          temp_file_path = temp_file.name

      loader = None
      if file_extension == ".pdf":
          loader = PyPDFLoader(temp_file_path)

      if loader:
          text.extend(loader.load())
          os.remove(temp_file_path)

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
  text_chunks = text_splitter.split_documents(text)

  # Create embeddings
  embeddings = load_embeddings()

  # Create vector store
  vector_store = load_vectorstore(text_chunks, embeddings)

  # Create the chain object
  chain = load_model(vector_store)

  display_chat_history(chain)


def main():
  # Initialize session state
  initialize_session_state()
  with st.container():
    # BrightSites Image and Title
    st.title("BrightSitesGPT")
    gpt_img =  Image.open('./img.png')
    st.image(gpt_img, width = 400)
  with st.container():
      st.caption(':green[Upload your PDF file/s to get started.]')

  # Initialize Streamlit
  st.sidebar.title("Document Processing")
  uploaded_files = st.sidebar.file_uploader("Upload files", accept_multiple_files=True)

  if uploaded_files:
    with st.spinner("Loading..."):
        processing(uploaded_files)

if __name__ == "__main__":
  main()

Overwriting BrightSitesGPT.py


In [ ]:
# Run the Streamlit application
!streamlit run BrightSitesGPT.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.143.241.171
npx: installed 22 in 3.156s
your url is: https://tall-symbols-enjoy.loca.lt
